# Eshelby Example

## Rotation functions

First we will define rotation functions for $Q_{ij}$ and $R_\sigma$

In [1]:
import numpy as np

def qij_x(theta):
    """
    rotation tensor about x-axis by some theta
    input: theta (angle in radians)
    output: qij (3x3 rotation tensor)
    """
    qij = np.array([[1,0,0],
                   [0,np.cos(theta),np.sin(theta)],
                   [0,-np.sin(theta),np.cos(theta)]])
    return qij

def qij_y(theta):
    """
    rotation tensor abo|ut y-axis by some theta
    input: theta (angle in radians)
    output: qij (3x3 rotation tensor)
    """
    qij = np.array([[np.cos(theta), 0, -np.sin(theta)],
                   [0,1,0],
                   [np.sin(theta),0,np.cos(theta)]])
    return qij

def qij_z(theta):
    """
    rotation tensor about z-axis by some theta
    input: theta (angle in radians)
    output: qij (3x3 rotation tensor)
    """
    qij = np.array([[np.cos(theta),np.sin(theta),0],
                   [-np.sin(theta),np.cos(theta),0],
                   [0,0,1]])
    return qij

def R_sigma(q):
    """
    rotation matrix for engineering notation given some rotation tensor, qij
    note: uses convention for sigma = [s11, s22, s33, s23, s13, s12]
    input: q (3x3 rotation tensor)
    output: R_s (6x6 rotation matrix)
    """
    R_s = np.array([[q[0,0]**2,q[0,1]**2,q[0,2]**2,2.*q[0,1]*q[0,2],2.*q[0,0]*q[0,2], 2.*q[0,0]*q[0,1]],
                       [q[1,0]**2,q[1,1]**2,q[1,2]**2,2.*q[1,1]*q[1,2],2.*q[1,0]*q[1,2], 2.*q[1,0]*q[1,1]],
                       [q[2,0]**2,q[2,1]**2,q[2,2]**2,2.*q[2,1]*q[2,2],2.*q[2,0]*q[2,2], 2.*q[2,0]*q[2,1]],
                       [q[1,0]*q[2,0], q[1,1]*q[2,1], q[1,2]*q[2,2],q[1,2]*q[2,1]+q[1,1]*q[2,2], q[1,2]*q[2,0]+q[1,0]*q[2,2], q[1,1]*q[2,0]+q[1,0]*q[2,1]],
                       [q[0,0]*q[2,0], q[0,1]*q[2,1], q[0,2]*q[2,2],q[0,2]*q[2,1]+q[0,1]*q[2,2], q[0,2]*q[2,0]+q[0,0]*q[2,2], q[0,1]*q[2,0]+q[0,0]*q[2,1]],
                       [q[0,0]*q[1,0], q[0,1]*q[1,1], q[0,2]*q[1,2],q[0,2]*q[1,1]+q[0,1]*q[1,2], q[0,2]*q[1,0]+q[0,0]*q[1,2], q[0,1]*q[1,0]+q[0,0]*q[1,1]]])
    return R_s

## Eshelby Tensor

Now we calculate parameters for the Eshelby Tensor

In [2]:
nu_m = 0.40 #matrix poisson's ratio
s = 15. #fiber aspect ratio
I1 = (2.*s/(s**2.-1.)**1.5)*(s*(s**2.-1.)**.5-np.arccosh(s))
Q = 3./(8.*(1-nu_m))
R = (1.-2*nu_m)/(8*(1-nu_m))
T = Q*(4.-3.*I1)/(3.*(s**2.-1.))
I3 = 4.-2.*I1

And now we find the Eshelby tensor.

In [3]:
S = np.zeros((6,6))
S[0,0] = Q + R*I1 + 0.75*T
S[1,1] = S[0,0]
S[2,2] = 4./3.*Q + R*I3 + 2*s**2*T
S[0,1] = Q/3 - R*I1 + 4.*T/3.
S[1,0] = S[0,1]
S[0,2] = -R*I1 - s**2*T
S[1,2] = S[0,2]
S[2,0] = -R*I3 - T
S[2,1] = S[2,0]
S[5,5] = Q/3. + R*I1 + T/4.
S[3,3] = 2.*R - I1*R/2. - (1.+s**2)*T/2.
S[4,4] = S[3,3]

We now need to express the Eshelby tensor into the $\pm 45^\circ$ directions.

Fibers are oriented at $\pm 45$ in the $x-y$ plane, but the local $x_3^\prime$ axis is what points in the fiber direction.

One set of transformations to achieve this state is first a rotation of $90^\circ$ about the x2-axis, followed by a rotation of  $\pm45^\circ$ about the new x1' axis. 

As we found from our example in Lecture 2 (slide 35), one effective Q for this transformation can be found by multiplying $Q^2 Q^1$. However, we wish to utilize the inverse transformation to get properties from the local coordinates to global, so we use $Q^{-1} = Q^T = Q^{1T} Q^{2T}$.

In [4]:
Q1 = qij_y(np.pi/2)
Q2_45 = qij_x(-np.pi/4) #plus 45 rotation (x is in -z direction)
Q2_m45 = qij_x(np.pi/4) #minus 45 rotation (x is in -z direction)

Q_45 = np.dot(Q1.T,Q2_45.T) #matrix multiplication in python
Q_m45 = np.dot(Q1.T,Q2_m45.T)

R_45 = R_sigma(Q_45)
R_m45 = R_sigma(Q_m45)

S_45 = np.dot(R_45,np.dot(S,R_45.T))
S_m45 = np.dot(R_m45,np.dot(S,R_m45.T))

In [5]:
#check rotations
#np.dot(Q_m45,[0,0,1])
def tensrot(Q,tens):
    temp = np.dot(Q,np.dot(tens,Q.T))
    return np.array([temp[0,0],temp[1,1],temp[2,2],temp[1,2],temp[0,2],temp[0,1]])

sig = np.array([[0,1,0],
             [1,1,0],
             [0,0,1]])

print np.round(np.dot(R_m45,[sig[0,0],sig[1,1],sig[2,2],sig[1,2],sig[0,2],sig[0,1]]),decimals=2)
print np.round(tensrot(Q_m45,sig),decimals=2) #this confirms that R_m45 behaves as expected

print np.round(np.dot(Q_m45,[0,0,1]),decimals=2) #this confirms Q_m45 rotates as expected

[ 1.    1.    0.   -0.71 -0.71  0.  ]
[ 1.    1.    0.   -0.71 -0.71  0.  ]
[ 0.71 -0.71  0.  ]


# Stiffness Matrix

Now we need to build the stiffness matrices for $C_f$ and $C_m$ to find the strain concentration tensors.
For simplicity, we will assume isotropic material properties.
If the matrix were not isotropic, we would need to use a different formula to calculate the Eshelby tensor, but that is not the case here.

In [6]:
Ef = 200. #GPa
nu_f = 0.2
Em = 10 #GPa
#nu_m defined above
Cf = np.zeros((6,6))
Cf[0:3,0:3] = nu_f*np.ones((3,3))
Cf = Cf + (1-2*nu_f)*np.eye(6)
Cf[3:,3:] = Cf[3:,3:]/2
Cf = Ef/((1+nu_f)*(1-2*nu_f))*Cf

Cm = np.zeros((6,6))
Cm[0:3,0:3] = nu_m*np.ones((3,3))
Cm = Cm + (1-2*nu_m)*np.eye(6)
Cm[3:,3:] = Cm[3:,3:]/2
Cm = Em/((1+nu_m)*(1-2*nu_m))*Cm

# Strain Concentration Tensor
We can now find the strain concentration tensors
$$
A^{45} = \left[I + S^{45} (C_m)^{-1}(C_f-C_m) \right]^{-1}
$$
$$
A^{-45} = \left[I + S^{-45} (C_m)^{-1}(C_f-C_m) \right]^{-1}
$$

In [7]:
A_45 = np.linalg.inv(np.eye(6)+np.dot(np.dot(S_45,np.linalg.inv(Cm)),(Cf-Cm)))
A_m45 = np.linalg.inv(np.eye(6)+np.dot(np.dot(S_m45,np.linalg.inv(Cm)),(Cf-Cm)))

## Effective Stiffness
And, finally, we can use the strain concentration tensors to calculate the effective stiffness.
$$
C = C_m + v^{45}(C_f-C_m)A^{45} + v^{-45}(C_f-C_m)A^{-45}
$$

In [8]:
v_45 = 0.1 #volume fraction of +45
v_m45 = 0.1 #volume fraction of -45
C = Cm + v_45*np.dot((Cf-Cm),A_45) + v_m45*np.dot((Cf-Cm),A_m45)

To interpret the stiffness, we convert it to compliance and find youngs modulus in each direction

In [9]:
S = np.linalg.inv(C)
print 1/S[0,0]
print 1/S[1,1]
print 1/S[2,2]

3.63028862005
3.63028862005
15.4129563847


In [12]:
np.round(C,decimals=2)

array([[ 26.19,  24.25,  16.5 ,   0.  ,   0.  ,   0.  ],
       [ 24.25,  26.19,  16.5 ,   0.  ,   0.  ,   0.  ],
       [ 16.48,  16.48,  26.2 ,   0.  ,   0.  ,   0.  ],
       [  0.  ,   0.  ,   0.  ,   1.52,   0.  ,   0.  ],
       [  0.  ,   0.  ,   0.  ,   0.  ,   1.52,   0.  ],
       [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,  13.2 ]])